In [30]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

# Define grayscale values for the 2x2 image
grayscale_values = [64, 128, 192, 255]

# Map grayscale values to angles
M = 256  # Number of colors
angles = [(np.pi * (value - 1)) / (2 * (M - 1)) for value in grayscale_values]

# Normalize the amplitudes
norm_factor = np.sqrt(sum(angle**2 for angle in angles))
amplitudes = [angle / norm_factor for angle in angles]

# Create a quantum circuit with 3 qubits (2 for pixel coordinates, 1 for grayscale)
n_qubits = 3
qc = QuantumCircuit(n_qubits)

# Encoding each amplitude into the circuit using controlled Ry gates
# Initial state |000> for pixel (0,0)
qc.ry(2 * np.arcsin(amplitudes[0]), 2)  # Apply Ry rotation on the grayscale qubit
qc.barrier()  # Add a barrier to separate the operations

# Controlled rotations for the other pixels
qc.x(0)  # Flip qubit 0 (first pixel coordinate)
qc.ry(2 * np.arcsin(amplitudes[1]), 2)  # Controlled rotation for pixel (0,1)
qc.barrier()  # Add a barrier
qc.x(0)  # Flip back qubit 0

qc.x(1)  # Flip qubit 1 (second pixel coordinate)
qc.ry(2 * np.arcsin(amplitudes[2]), 2)  # Controlled rotation for pixel (1,0)
qc.barrier()  # Add a barrier
qc.x(1)  # Flip back qubit 1

qc.x([0, 1])  # Flip both qubits for pixel (1,1)
qc.ry(2 * np.arcsin(amplitudes[3]), 2)  # Controlled rotation for pixel (1,1)
qc.barrier()  # Add a barrier
qc.x([0, 1])  # Flip back both qubits

# Visualize the quantum circuit
print("Quantum Circuit:")
print(qc)

# Simulate the quantum state using AerSimulator
simulator = AerSimulator(method="statevector")  # Ensure statevector simulation
transpiled_circuit = transpile(qc, simulator)  # Transpile for the simulator
result = simulator.run(transpiled_circuit).result()
statevector = result.get_statevector()  # Correct method to retrieve the statevector

# Print the final statevector
print("\nFinal Statevector:")
for i, amp in enumerate(statevector):
    print(f"State |{i:03b}>: Amplitude = {amp}")


Quantum Circuit:
                     ░      ┌───┐      ░     ┌───┐     ░     ┌───┐           ░ »
q_0: ────────────────░──────┤ X ├──────░─────┤ X ├─────░─────┤ X ├───────────░─»
                     ░      └───┘      ░     ├───┤     ░     ├───┤     ┌───┐ ░ »
q_1: ────────────────░─────────────────░─────┤ X ├─────░─────┤ X ├─────┤ X ├─░─»
     ┌─────────────┐ ░ ┌─────────────┐ ░ ┌───┴───┴───┐ ░ ┌───┴───┴────┐└───┘ ░ »
q_2: ┤ Ry(0.36409) ├─░─┤ Ry(0.74717) ├─░─┤ Ry(1.162) ├─░─┤ Ry(1.6364) ├──────░─»
     └─────────────┘ ░ └─────────────┘ ░ └───────────┘ ░ └────────────┘      ░ »
«     ┌───┐
«q_0: ┤ X ├
«     ├───┤
«q_1: ┤ X ├
«     └───┘
«q_2: ─────
«          


QiskitError: 'No statevector for experiment "None"'

In [29]:
# Simulate the quantum state using AerSimulator
simulator = AerSimulator(method="statevector")  # Ensure statevector simulation
transpiled_circuit = transpile(qc, simulator)  # Transpile for the simulator
result = simulator.run(transpiled_circuit).result()
statevector = result.get_counts(qc)  # Explicitly retrieve statevector
print(result)
# Print the final statevector
print("\nFinal Statevector:")
for i, amp in enumerate(statevector):
    print(f"State |{i:03b}>: Amplitude = {amp}")

QiskitError: 'No counts for experiment "<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x72375019a510>"'

In [27]:
counts_statevector

<bound method Result.get_counts of Result(backend_name='aer_simulator_statevector', backend_version='0.15.1', qobj_id='', job_id='945dd8b7-5779-4147-a491-7fa2d83c98b9', success=True, results=[ExperimentResult(shots=1000, success=True, meas_level=2, data=ExperimentResultData(), header=QobjExperimentHeader(creg_sizes=[], global_phase=0.0, memory_slots=0, n_qubits=3, name='circuit-192', qreg_sizes=[['q', 3]], metadata={}), status=DONE, seed_simulator=3789408716, metadata={'time_taken': 5.4071e-05, 'num_bind_params': 1, 'required_memory_mb': 1, 'input_qubit_map': [], 'method': 'statevector', 'device': 'CPU', 'num_qubits': 0, 'parallel_state_update': 36, 'active_input_qubits': [], 'num_clbits': 0, 'remapped_qubits': False, 'parallel_shots': 1, 'runtime_parameter_bind': False, 'max_memory_mb': 64164, 'measure_sampling': False, 'batched_shots_optimization': False}, time_taken=5.4071e-05)], date=2024-11-18T19:56:35.212951, status=COMPLETED, header=None, metadata={'time_taken_parameter_binding'